In [4]:
#Imports
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover

#Create a spark session
spark = SparkSession.builder.appName("SpookyAuthorIdentification").getOrCreate()

Stage 0: Import Data

In [5]:
#Load training data into a data frame
train_df = spark.read.csv('Datasets/train.csv', header=True, inferSchema=True)

#Verify
train_df.printSchema()
train_df.show(5)


root
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- author: string (nullable = true)

+-------+--------------------+------+
|     id|                text|author|
+-------+--------------------+------+
|id26305|This process, how...|   EAP|
|id17569|It never once occ...|   HPL|
|id11008|In his left hand ...|   EAP|
|id27763|How lovely is spr...|   MWS|
|id12958|Finding nothing e...|   HPL|
+-------+--------------------+------+
only showing top 5 rows



In [6]:
#Load test data into a data frame
test_df = spark.read.csv('Datasets/test.csv', header=True, inferSchema=True)

#Verify
test_df.printSchema()
test_df.show(5)

root
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)

+-------+--------------------+
|     id|                text|
+-------+--------------------+
|id02310|Still, as I urged...|
|id24541|If a fire wanted ...|
|id00134|And when they had...|
|id27757|While I was think...|
|id04081|I am not sure to ...|
+-------+--------------------+
only showing top 5 rows



In [7]:
#Print num rows and cols in the training set
train_row_count = train_df.count()
train_column_count = len(train_df.columns)
print(f"Training Data - Rows: {train_row_count}, Columns: {train_column_count}")

Training Data - Rows: 19579, Columns: 3


In [8]:
#Print num rows and cols in the test set
test_row_count = test_df.count()
test_column_count = len(test_df.columns)
print(f"Test Data - Rows: {test_row_count}, Columns: {test_column_count}")

Test Data - Rows: 8392, Columns: 2


In [9]:
#Check for missing values (There are none)
train_df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in train_df.columns]).show()
test_df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in test_df.columns]).show()

+---+----+------+
| id|text|author|
+---+----+------+
|  0|   0|     0|
+---+----+------+

+---+----+
| id|text|
+---+----+
|  0|   0|
+---+----+



Stage 1: Data Preparation - Exploratory data anlysis and text mining pre-processing

In [10]:
#Create sentence length column
train_df = train_df.withColumn("sentence_length", F.length(F.col("text")))
train_df.select("sentence_length").describe().show()

+-------+------------------+
|summary|   sentence_length|
+-------+------------------+
|  count|             19579|
|   mean|139.99765054395016|
| stddev|101.25452331007808|
|    min|                 5|
|    max|              3682|
+-------+------------------+



In [11]:
#Use tokenizer to tokenize the text
tokenizer = Tokenizer(inputCol="text", outputCol="words")
tokenized_train_df = tokenizer.transform(train_df)

#Print tokenized output
tokenized_train_df.select("text", "words").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                   |words                                                                                                                                                                                                                                                              

In [12]:
#Create an instance of StopWordsRemover and use it to filter the tokens and remove stop words
#The new filtered tokens are placed in a new column "filtered_words"
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
cleaned_train_df = remover.transform(tokenized_train_df)

#Print the cleaned output
cleaned_train_df.select("words", "filtered_words").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words                                                                                                                                                                                                                                                                            |filtered_words                                                                                                                                                                              |
+---------------------------------------------------------------------

Stage 2: Feature Extraction

In [15]:
#Set up Countvectorizer to count the occurences of each word in the cleaned data
vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="raw_features")
vectorizer_model = vectorizer.fit(cleaned_train_df)

#Transform the cleaned text into raw features
raw_features_df = vectorizer_model.transform(cleaned_train_df)

In [18]:
#Set up the IDF transformer
idf = IDF(inputCol="raw_features", outputCol="tfidf_features")

#Fit the IDF model and transform raw features
idf_model = idf.fit(raw_features_df)
tfidf_df = idf_model.transform(raw_features_df)

tfidf_df.select("tfidf_features").show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tfidf_features                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             